### Project title:

Predicting Knowledge Levels 

Group 40 Members: Sam Kao, Namya Sharma, Lucy Duan, Jason Zhou

### Introduction:

A person’s level of knowledge cannot be evaluated unilaterally, and different people with different knowledge levels could be better assigned an appropriate job in a firm or in a research group, so that they will do their own work efficiently. The User Knowledge datasat covers six variables: STG (The degree of study time for goal object materials), SCG (The degree of repetition number of user for goal object materials), STR (The degree of study time of user for related objects with goal object), LPR (The exam performance of user for related objects with goal object), PEG (The exam performance of user for goal objects), and UNS (an individual’s knowledge level). There are a total of 403 observations and the first five variables are all numerical doubles, whereas UNS is a categorical variable. 
As the time passes, the global job market becomes more competitive than before. Thousands of people with differing strengths compete for the same job; therefore, we need a model to select applicants most efficiently. Such a model can alleviate stress from a corporation's recruitment team.

Question: What will the knowledge level (UNS) of a user be (very low, low, middle or high) based on their length of study of the goal objects (STG), and their exam performance of goal object (PEG)?

### Preliminary exploratory data analysis:

In [ ]:
library(tidyverse)
library(readxl)
library(dplyr)
library(gridExtra)
options(repr.matrix.max.rows = 6)

In [ ]:
# reading the data
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00257/Data_User_Modeling_Dataset_Hamdi%20Tolga%20KAHRAMAN.xls"
download.file(url, "data.xls")
training_data <- read_excel("data.xls", sheet = 2) # sheet 2 => training data

# select only the columns we need and making our class as a factor
training_data <- training_data %>%
    select(1:6) %>%
    mutate(UNS = as_factor(UNS))

training_data

In [ ]:
# summarize the training data, counting how many occurences of each class 
num_of_class <- training_data %>%
    mutate(UNS = factor(UNS, levels = c("High", "Middle", "Low", "very_low"))) %>%
    count(UNS)

num_of_class

In [ ]:
#the means of the predictor variables you plan to use in your analysis
predictors_mean <- training_data %>%
                  summarize (mean_STG = mean(STG, na.rm = TRUE),
                             mean_PEG = mean(PEG, na.rm = TRUE))
predictors_mean

In [ ]:
#how many rows have missing data
missing_data <- training_data %>% 
        filter(is.na(STG), is.na(PEG), is.na(SCG), is.na(STR), is.na(LPR), is.na(UNS))
missing_data_count <- nrow(missing_data)
missing_data_count

In [ ]:
# plot STG vs PEG and coloring the class of UNS
# STG => The degree of study time for goal object materails
# PEG => The exam performance of user for goal objects
options(repr.plot.width = 10, repr.plot.height = 7) 
plot <- training_data %>%
    mutate(UNS = factor(UNS, levels = c("High", "Middle", "Low", "very_low"))) %>%
    ggplot(aes(x = STG, y = PEG, color = UNS)) +
    geom_point() +
    labs(x = "Degree of Study Time", y="Exam performance", color= "The knowledge level of user")+
    ggtitle('STG vs PEG')+
    theme(text = element_text(size=20))

plot

In [ ]:
#compare the distributions of each of the predictor variables you plan to use in your analysis
#the distribution of exam performace of goal objects

options(repr.plot.width = 10, repr.plot.height = 7) 
distr_PEG <- training_data %>%
    mutate(UNS = factor(UNS, levels = c("High", "Middle", "Low", "very_low"))) %>% 
    ggplot(aes(x = PEG, fill = UNS)) +
    geom_histogram(binwidth = .01)+
    labs(x = "Exam performance of goal objects", fill = "The knowledge level of user")+
    ggtitle("The distribution of exam performace of goal objects")+
    theme(text = element_text(size=20))+
    facet_grid(rows = vars(UNS))

distr_PEG

In [ ]:
#compare the distributions of each of the predictor variables you plan to use in your analysis
#The distribution of the degree of study time for goal object materails (STG)
options(repr.plot.width = 10, repr.plot.height = 7) 
distr_STG <- training_data %>%
    mutate(UNS = factor(UNS, levels = c("High", "Middle", "Low", "very_low"))) %>%
    ggplot(aes(x = STG, fill = UNS)) +
    geom_histogram(binwidth = .01)+
    labs(x = "Study time for goal object materails", fill = "The knowledge level of user")+
    ggtitle("The distribution of study time for goal object materails")+
    theme(text = element_text(size= 20))+
    facet_grid(rows = vars(UNS))

distr_STG


In [ ]:
#compare the distributions of each of the predictor variables you plan to use in your analysis
library(cowplot)
options(repr.plot.width = 19, repr.plot.height = 8)
plot_grid(distr_PEG, distr_STG)
options(repr.plot.width = 19, repr.plot.height = 8)

### Methods:

We will use STG (The degree of study time for goal object materials) and PEG (The exam performance of users for goal objects) to find the relationships with UNS (an individual’s knowledge level). We decided to use these variables since we believe time spent studying for goal object and its respective exam performance are the best predictors for an individual’s knowledge level as they offer less random outliers, and are directly correlated to knowledge levels. We will select these three columns and plot STG vs PEG with the four UNS classes plotting as different colours. A scatter plot with four colours will allow us to visualize the relationship between the three variables better. We will then apply a k-nearest neighbour model to our training set and use STG and PEG to predict an individual’s UNS. We will also tune our model to select the best ‘k’ value for our model by performing cross validation before applying our fitted model to our testing dataset, from which we can determine the accuracy of it.


### Expected outcomes and significance:

We expect STG and PEG to be the two best predictors to classify the UNS of a new individual, and thus produce a model with relatively good accuracy in relation to the context and application of our model. After preliminary analysis, we expect PEG to play a larger role than STG in UNS prediction since graphically, those with higher PEG values tend to be classified as “high” in terms of UNS. These findings can allow us to  predict an individual’s knowledge level based on degree of study time and exam performance in the real world.  A prediction of the reversal could also be made. In the future, it would be interesting to also investigate the impact of other variables such as STR and LPR on classifying knowledge level.